# Topic Modelling on Program Source Code
---
By Kishalay Banerjee, Dan Jones and Sam Harding

In [ ]:
import pandas
import numpy
import pickle
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [ ]:
numpy.random.seed(0xD00D5) 

## Generating the Dataset

TODO: Import dataset.py and explain how it's used + what it does.

TODO: Add Sam's scrub function in here?

In [ ]:
minimal_dataset = pandas.read_csv("../data/dataset.csv.gz", header=None, names=['repo', 'language', 'documents'])
minimal_dataset.head()

## Topic Modelling on Individual Source Files

Basically done, just need to copy it over. Maybe run on the bigger dataset?

TODO:
  - Copy work from documentation/daniel-jones.ipynb
  - Add visualisation with pyldavis

For our purposes, common words are important and rare words aren't. So we shouldn't use tf-idf as a metric, bag-of-words makes more sense. (TODO: Maybe: "Similarly, filter out words that don't occur very often").


In [ ]:
tf_vectorizer = CountVectorizer(stop_words=None)
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()

We have four programming languages, try to use LDA to determine these four programming languages.

In [ ]:
number_of_languages = 4

lda = LatentDirichletAllocation(n_topics=number_of_languages,  n_jobs=1)
model = lda.fit(tf)

In [ ]:
with open('../data/minimal_lda_model.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(model, f)

In [ ]:
with open('../data/minimal_lda_model.pickle', 'rb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    model = pickle.load(f)

In [ ]:
pyLDAvis.sklearn.prepare(model, tf, tf_vectorizer)

Reference: https://nbviewer.jupyter.org/github/bmabey/pyLDAvis/blob/master/notebooks/sklearn.ipynb

## Topic Modelling on Programming Language Mixtures
Keypoint: topics are programming languages, file with mixture of programming languages, identify which is which.

Applicability to cyber-security: identifying malware embedded within normal programs (shellcode).

### Combining Source Files per Repository

TODO:
  - Copy work Sam's
  
  
### LDA Model

TODO:
  - Write model
  - Add visualisation with pyldavis
 

## Identifying more general topics/themes in program source code (DAN)

Hypothesis: Using tf-idf rather than bag-of-words as an input to LDA will prioritise rare words. In the case of source code, this means programming language keywords (an identifying feature of programming languages) are deprioritised, and so a more human idea of topics may emerge. 

We can use repo-list.json and the repo-ids to map the github topics/tags to each repo. Might be a small/easy task to compare against the programming langauge identification.

## Measuring the Efficacy of Topic Models (WRITEUP: WHO?)

Main question: How do we evaluate how well a topic (from LDA for example) represents a meaningful "topic" or theme?

TODO: do some research on this??? There must be some papers etc that try to formalise this that we can borrow ideas from?

Paper dump:
  - Looks like a good summary paper: http://www.aclweb.org/anthology/E14-4005 Find more papers from this ones references?
    - " KL-divergence (Li and McCallum, 2006; Wang et al., 2009; Newman et al., 2009), cosine measure (He et al., 2009; Ramage et al., 2009) and the average Log Odds Ratio (Chaney and Blei, 2012). "
    - "Kim and Oh (2011) also applied  the  cosine  measure  and  KL-Divergence which were compared with four other measures: Jaccard’s Coefficient, Kendall’s τ coefficient, Discount  Cumulative  Gain  and  Jensen  Shannon  Divergence (JSD)."
  - Cool name haven't read it: http://papers.nips.cc/paper/3700-reading-tea-leaves-how-humans-interpret-topic-models.pdf

### Reading Tea Leaves Paper
http://papers.nips.cc/paper/3700-reading-tea-leaves-how-humans-interpret-topic-models.pdf

### Word Overlap

I think this is used as a baseline measure in the summary paper above (http://www.aclweb.org/anthology/E14-4005). Should be a quick implementation so worth a try.

### Jaccard Index

From the papers above this seems to have been used relatively often for _linking machine-generated topics to human topics_ and so maybe this is a good application for it. Apparently explored here "https://link.springer.com/chapter/10.1007/978-3-642-19437-5_13" but I haven't read it.


### Comparing topic mixtures generated by model with measured mixtures, i.e. how good are our models?
TODO: better title needed

Idea:
  - save the actual % of each program langauge per repo
  - Then try to use LDA model to tell us "I believe repo <x> is 10% Topic 1, 20% Topic 2 etc". 
  - Use analysis from above two sections to create a "most likely mapping from lda topic to programming language".
  - rate our models

Here we can do cross-validation etc.